In [ ]:
from typing import TypedDict , Annotated
from langgraph.graph import StateGraph, START , END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
import os


In [82]:
load_dotenv()

True

In [83]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="gemini-1.5-flash",
    model_provider="google_genai",
    api_key=os.getenv("GEMINI_API"),
    temperature=0.2
)

response = llm.invoke("What is the capital of France?")
print(response.content)


Paris


In [84]:
class State(TypedDict):
    messages: Annotated[list , add_messages]

def chatbot(state: State) -> State:
    return {
       "messages" : [llm.invoke(state['messages'])]
    }

In [85]:
builder = StateGraph(State)
builder.add_node("chatbot" , chatbot)

builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

graph = builder.compile()

In [86]:
message = {
    "role": "user",
    "content": "What is the capital of France?"
}

response = graph.invoke(
    {
        "messages": [message]
    }
)

print(response)

{'messages': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}, id='3f3d1725-4415-4e5e-9f01-093e13f0de9b'), AIMessage(content='Paris', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--55679520-ccb7-43f5-83a2-b1607b8232ee-0', usage_metadata={'input_tokens': 7, 'output_tokens': 2, 'total_tokens': 9, 'input_token_details': {'cache_read': 0}})]}
